In [4]:
!pip install graphviz
!pip install web3

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 47 kB 1.9 MB/s             


In [3]:
from tree import Node, insert, vis

root = Node(1)
for key in range(2, 2**5):
    insert(root, key)

vis(root)

In [9]:
from pprint import pprint
import json
from web3 import Web3

from lib import write_8_bits
from game_types import VMStatus
from block import Block
from game import (
    FaultDisputeGame, 
    VM,
    ABSOLUTE_PRESTATE,
    MAX_GAME_DEPTH,
    SPLIT_DEPTH, MAX_CLOCK_DURATION,
    CLOCK_EXTENSION,
    CHALLENGE_PERIOD
)

def hash_str(s: str) -> str:
    vals = [s]
    types = ['string']
    # keccak256(abi.encodePacked(...))
    return Web3.solidity_keccak(types, vals).hex()

block = Block()
vm = VM()
game = FaultDisputeGame(vm = VM)
game.initialize(
    l2_root = hash_str("l2 root"),
    l2_block_num = 1,
    root_claim = hash_str("root"),
    game_creator = "defender",
    bond = 0,
    block_timestamp = block.timestamp
)

block.inc()
game.move(
    disputed = hash_str("root"), 
    challenge_idx = 0, 
    claim = hash_str("2,0"), 
    is_attack = True, 
    msg_value = 0, 
    block_timestamp = block.timestamp,
    msg_sender = "challenger",
)

block.inc()
game.move(
    disputed = hash_str("2,0"), 
    challenge_idx = 1, 
    claim = hash_str("4,0"), 
    is_attack = True, 
    msg_value = 0, 
    block_timestamp = block.timestamp,
    msg_sender = "defender",
)

block.inc()
game.move(
    disputed = hash_str("4,0"), 
    challenge_idx = 2, 
    claim = write_8_bits(hash_str("8,0"), VMStatus.INVALID.value), 
    is_attack = True, 
    msg_value = 0, 
    block_timestamp = block.timestamp,
    msg_sender = "challenger",
)

block.warp(MAX_CLOCK_DURATION + 2)
game.resolve_claim(3, 0, block_timestamp = block.timestamp)
game.resolve_claim(2, 0, block_timestamp = block.timestamp)
game.resolve_claim(1, 0, block_timestamp = block.timestamp)
game.resolve_claim(0, 0, block_timestamp = block.timestamp)
game.resolve()
game.status


<GameStatus.CHALLENGER_WINS: 1>

In [ ]:

###  no challenge -> defender wins ###
# block.warp(MAX_CLOCK_DURATION)
# game.resolve_claim(0, 0, block_timestamp = block.timestamp
# game.resolve()
# game.status

### attack -> challenger wins ###
# block.inc()
# game.move(
#     disputed = hash_str("root"), 
#     challenge_idx = 0, 
#     claim = hash_str("2,0"), 
#     is_attack = True, 
#     msg_value = 0, 
#     block_timestamp = block.timestamp,
#     msg_sender = "challenger",
# )

# # Each party (defender and challenger) gets MAX_CLOCK_DURATION
# # Need to wait for both parties to use MAX_CLOCK_DURATION
# block.warp(MAX_CLOCK_DURATION + 1)
# game.resolve_claim(1, 0, block_timestamp = block.timestamp)
# game.resolve_claim(0, 0, block_timestamp = block.timestamp)
# game.resolve()
# game.status

### attack -> defend -> defender wins ###
# block.inc()
# game.move(
#     disputed = hash_str("root"), 
#     challenge_idx = 0, 
#     claim = hash_str("2,0"), 
#     is_attack = True, 
#     msg_value = 0, 
#     block_timestamp = block.timestamp,
#     msg_sender = "challenger",
# )

# block.inc()
# game.move(
#     disputed = hash_str("2,0"), 
#     challenge_idx = 1, 
#     claim = hash_str("6,0"), 
#     is_attack = False, 
#     msg_value = 0, 
#     block_timestamp = block.timestamp,
#     msg_sender = "defender",
# )

# block.warp(MAX_CLOCK_DURATION + 1)
# game.resolve_claim(2, 0, block_timestamp = block.timestamp)
# game.resolve_claim(1, 0, block_timestamp = block.timestamp)
# game.resolve_claim(0, 0, block_timestamp = block.timestamp)
# game.resolve()
# game.status

### attack -> attack -> defender wins ###
# block.inc()
# game.move(
#     disputed = hash_str("root"), 
#     challenge_idx = 0, 
#     claim = hash_str("2,0"), 
#     is_attack = True, 
#     msg_value = 0, 
#     block_timestamp = block.timestamp,
#     msg_sender = "challenger",
# )

# block.inc()
# game.move(
#     disputed = hash_str("2,0"), 
#     challenge_idx = 1, 
#     claim = hash_str("4,0"), 
#     is_attack = True, 
#     msg_value = 0, 
#     block_timestamp = block.timestamp,
#     msg_sender = "defender",
# )

# block.warp(MAX_CLOCK_DURATION + 1)
# game.resolve_claim(2, 0, block_timestamp = block.timestamp)
# game.resolve_claim(1, 0, block_timestamp = block.timestamp)
# game.resolve_claim(0, 0, block_timestamp = block.timestamp)
# game.resolve()
# game.status

### attack -> defend -> defend -> challenger wins ###
block.inc()
game.move(
    disputed = hash_str("root"), 
    challenge_idx = 0, 
    claim = hash_str("2,0"), 
    is_attack = True, 
    msg_value = 0, 
    block_timestamp = block.timestamp,
    msg_sender = "challenger",
)

block.inc()
game.move(
    disputed = hash_str("2,0"), 
    challenge_idx = 1, 
    claim = hash_str("6,0"), 
    is_attack = False, 
    msg_value = 0, 
    block_timestamp = block.timestamp,
    msg_sender = "defender",
)

block.inc()
game.move(
    disputed = hash_str("6,0"), 
    challenge_idx = 2, 
    claim = hash_str("14,0"), 
    is_attack = False, 
    msg_value = 0, 
    block_timestamp = block.timestamp,
    msg_sender = "challenger",
)

block.warp(MAX_CLOCK_DURATION + 2)
game.resolve_claim(3, 0, block_timestamp = block.timestamp)
game.resolve_claim(2, 0, block_timestamp = block.timestamp)
game.resolve_claim(1, 0, block_timestamp = block.timestamp)
game.resolve_claim(0, 0, block_timestamp = block.timestamp)
game.resolve()
game.status

In [7]:
from game_types import VMStatus

t = VMStatus.INVALID.value
int(t)

1